In [1]:
from matplotlib import pyplot as plt
from tqdm import tqdm
import os
from pathlib import Path, PurePath
import csv
import pandas as pd
import numpy as np

import timeit

import matplotlib_inline.backend_inline
matplotlib_inline.backend_inline.set_matplotlib_formats("retina")

from sklearn.model_selection import LeaveOneGroupOut
from sklearn.model_selection import StratifiedKFold

from sklearn.feature_selection import RFECV
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
from sklearn.inspection import permutation_importance

import catboost as cb
from catboost import CatBoostClassifier

import xgboost as xgb
from xgboost import XGBClassifier

import lightgbm

In [2]:
rlist = []
records = PurePath(Path(os.getcwd()).parents[1], Path('mit-bih-dataframes/subject_list.csv'))
with open(records) as rfile:
    recordreader = csv.reader(rfile, delimiter=' ', quotechar='|')
    for row in recordreader:
        rlist.append(row[0])

In [3]:
performance_dict = {
    "Model name": [],
    "Avg Accuracy": [],
    "Std Accuracy": [],
    "Sensitivity": [],
    "Specificity": [],
    "Precision": [],
    "F1 score": [],
    "Run time": [],
    "TPS": []
}

moving_accuracy = {}

In [4]:
feature_dfs = {}
for record in tqdm(rlist):
    feature_dfs[record] = pd.read_parquet(PurePath(Path(os.getcwd()).parents[1], Path('mit-bih-time-features/' + record+ '.parquet')))

combined_features = pd.concat([feature_dfs[key][1:] for key in feature_dfs])

100%|██████████| 23/23 [00:00<00:00, 261.71it/s]


In [5]:
X = combined_features[['StoS', 'StoR', 'StoL', 'RtoS', 'RtoR', 'RtoL', 'LtoS', 'LtoR', 'LtoL', 'std', 'cov', 'range', 'rrInt_var', 'rmean_var', 'rmssd', 'mad', 'iqr']]
y = combined_features['mappedLabel'].map({"Non-Afib": 0, "Afib": 1})
groups = combined_features['subjectID'].astype('int64')

logo = LeaveOneGroupOut()
splits = list(logo.split(X, y, groups=groups))
print(splits)

[(array([  7315,   7316,   7317, ..., 187554, 187555, 187556]), array([   0,    1,    2, ..., 7312, 7313, 7314])), (array([     0,      1,      2, ..., 187554, 187555, 187556]), array([ 7315,  7316,  7317, ..., 17613, 17614, 17615])), (array([     0,      1,      2, ..., 187554, 187555, 187556]), array([17616, 17617, 17618, ..., 24243, 24244, 24245])), (array([     0,      1,      2, ..., 187554, 187555, 187556]), array([24246, 24247, 24248, ..., 31367, 31368, 31369])), (array([     0,      1,      2, ..., 187554, 187555, 187556]), array([31370, 31371, 31372, ..., 39328, 39329, 39330])), (array([     0,      1,      2, ..., 187554, 187555, 187556]), array([39331, 39332, 39333, ..., 49603, 49604, 49605])), (array([     0,      1,      2, ..., 187554, 187555, 187556]), array([49606, 49607, 49608, ..., 58525, 58526, 58527])), (array([     0,      1,      2, ..., 187554, 187555, 187556]), array([58528, 58529, 58530, ..., 64634, 64635, 64636])), (array([     0,      1,      2, ..., 187554, 

In [6]:
selected_splits = []

for train, test in tqdm(splits):
    X_train = X.iloc[train]
    y_train = y.iloc[train]

    X_test = X.iloc[test]
    y_test = y.iloc[test]

    feature_selection_groups = combined_features.iloc[train]['subjectID'].astype('int64')

    cv = LeaveOneGroupOut()
    feature_selection_splits = list(cv.split(X_train, y_train, groups=feature_selection_groups))

    clf = XGBClassifier(n_estimators=50, learning_rate = 0.1, verbose=None,
                max_depth = 3, eval_metric='logloss', tree_method='gpu_hist')

    rfecv = RFECV(
        estimator=clf,
        step=1,
        cv=feature_selection_splits,
        scoring="accuracy",
        n_jobs=6
    )
    rfecv.fit(X_train, y_train)

    tqdm.write(f"Optimal number of features: {rfecv.n_features_}")
    tqdm.write(f"The features are: {rfecv.get_feature_names_out()}")

    selected_splits.append((X_train[rfecv.get_feature_names_out()], y_train, X_test[rfecv.get_feature_names_out()], y_test))

  4%|▍         | 1/23 [00:37<13:53, 37.88s/it]

Optimal number of features: 11
The features are: ['StoS' 'StoR' 'StoL' 'RtoL' 'LtoS' 'LtoR' 'LtoL' 'cov' 'rrInt_var' 'mad'
 'iqr']


  9%|▊         | 2/23 [01:14<12:58, 37.09s/it]

Optimal number of features: 12
The features are: ['StoS' 'StoR' 'StoL' 'RtoL' 'LtoS' 'LtoR' 'LtoL' 'cov' 'range'
 'rrInt_var' 'mad' 'iqr']


 13%|█▎        | 3/23 [01:50<12:16, 36.85s/it]

...

Optimal number of features: 12
The features are: ['StoS' 'StoR' 'StoL' 'RtoL' 'LtoS' 'LtoR' 'LtoL' 'cov' 'range'
 'rrInt_var' 'mad' 'iqr']


 96%|█████████▌| 22/23 [13:09<00:35, 35.26s/it]

Optimal number of features: 10
The features are: ['StoS' 'StoR' 'StoL' 'RtoL' 'LtoS' 'LtoR' 'LtoL' 'cov' 'mad' 'iqr']


100%|██████████| 23/23 [13:44<00:00, 35.85s/it]

Optimal number of features: 11
The features are: ['StoS' 'StoR' 'StoL' 'RtoL' 'LtoS' 'LtoR' 'LtoL' 'cov' 'rrInt_var' 'mad'
 'iqr']


In [7]:
# CatBoost
start_time = timeit.default_timer() #defines start time so computational time can be calculated

acc_score = []
Truth = []
Output = []
 
for X_train, y_train, X_test, y_test in tqdm(selected_splits):
    # Create CatBoost model
    model = CatBoostClassifier(iterations=500,
                           depth=10,
                           learning_rate=0.1,
                           loss_function='Logloss',
                           task_type="GPU")
    # train the model
    model.fit(X_train, y_train, verbose=False)
    
    # make the prediction using the resulting model
    pred_values = model.predict(X_test)
    #preds_proba = model.predict_proba(test_data)
    #print("class = ", preds_class)
    #print("proba = ", preds_proba)
     
    acc = accuracy_score(y_test, pred_values)
    acc_score.append(acc)
    
    Truth.extend(y_test.values.reshape(y_test.shape[0])); ## it is a list
    Output.extend(pred_values); ## it is a list  
    
    """
    #print(model.feature_importances_)
    Importance = pd.DataFrame({'Importance':(model.feature_importances_*100)[0:10]}, 
                          index = (X_train.columns)[0:10])
    Importance.sort_values(by = 'Importance', 
                       axis = 0, 
                       ascending = True).plot(kind = 'barh', 
                                              color = 'r')
    plt.xlabel('Variable Importance')
    plt.gca().legend_ = None
    """
    #plt.savefig('plot1.png')


elapsed = timeit.default_timer() - start_time #gives total computation time
print("---Run time is %s seconds ---" % elapsed) #prints computation time
print()

print('Accuracy of each fold: \n {}'.format(acc_score))
print()
print('Avg accuracy : \n{}'.format(np.mean(acc_score))); 
print()
print('Std of accuracy : \n{}'.format(np.std(acc_score)))

cm = confusion_matrix(Truth, Output)
print(cm)
print("classification report: ")
print(classification_report(Truth, Output))

sensitivity = cm[0][0]/(cm[0][0]+cm[0][1])
specificity = cm[1][1]/(cm[1][0]+cm[1][1])
precision = (cm[0][0])/(cm[0][0]+cm[1][0])
f1_score = (2*precision*sensitivity)/(precision+sensitivity)

print(sensitivity)
print(specificity)
print(precision)
print(f1_score)

performance_dict['Avg Accuracy'].append(np.mean(acc_score))
performance_dict['Std Accuracy'].append(np.std(acc_score))
performance_dict['Sensitivity'].append(sensitivity)
performance_dict['Specificity'].append(specificity)
performance_dict['Precision'].append(precision)
performance_dict['F1 score'].append(f1_score)
performance_dict['Run time'].append(elapsed)

100%|██████████| 23/23 [02:09<00:00,  5.62s/it]


---Run time is 129.29741582599854 seconds ---

Accuracy of each fold: 
 [0.9171565276828435, 0.9235025725657703, 0.9933634992458522, 0.9483436271757439, 0.9968596909935937, 0.9671046228710463, 0.9113427482627213, 0.9939433622524144, 0.9471971066907776, 0.9287884793235566, 0.9707774506596881, 0.9887679281147399, 0.8760348583877996, 0.907906834201655, 0.719501246882793, 0.9842082799829279, 0.9854328753517629, 0.9834836918806384, 0.7694179679578179, 0.9820295983086681, 0.991012154018997, 0.9785563273935367, 0.9949525540076721]

Avg accuracy : 
0.9417253914875224

Std of accuracy : 
0.06992808633824256
[[96528  6357]
 [ 5546 79126]]
classification report: 
              precision    recall  f1-score   support

           0       0.95      0.94      0.94    102885
           1       0.93      0.93      0.93     84672

    accuracy                           0.94    187557
   macro avg       0.94      0.94      0.94    187557
weighted avg       0.94      0.94      0.94    187557

0.9382125674

In [8]:
# XGBoost

start_time = timeit.default_timer() #defines start time so computational time can be calculated

acc_score = []
Truth = []
Output = []
 
for X_train, y_train, X_test, y_test in tqdm(selected_splits):
    a = y_test.to_numpy() # s.values (pandas<0.24)
    if (a[0] == a).all():
        continue

    model = XGBClassifier(n_estimators=200, learning_rate = 0.1, verbose=None,
                max_depth = 6, eval_metric='logloss', tree_method="exact")
    
    model.fit(X_train,y_train)

    pred_values = model.predict(X_test)
    pred_prob = model.predict_proba(X_test)[:, 1]
     
    acc = accuracy_score(y_test, pred_values)
    acc_score.append(acc)
    
    Truth.extend(y_test.values.reshape(y_test.shape[0])); ## it is a list
    Output.extend(pred_values); ## it is a list 
    
    """
    #print(model.feature_importances_)
    Importance = pd.DataFrame({'Importance':(model.feature_importances_*100)[0:10]}, 
                          index = (X_train.columns)[0:10])
    Importance.sort_values(by = 'Importance', 
                       axis = 0, 
                       ascending = True).plot(kind = 'barh', 
                                              color = 'r')
    plt.xlabel('Variable Importance')
    plt.gca().legend_ = None
    plt.show()
    #plt.savefig('plot1.png')
    
    ##################################################
    #https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html
    #Compute Area Under the Receiver Operating Characteristic Curve (ROC AUC) from prediction scores.
    print("roc_auc_score", roc_auc_score(y_test, pred_prob))
    
    # calculate the fpr and tpr for all thresholds of the classification
    #fpr, tpr, threshold = metrics.roc_curve(y_test, pred_prob)
    #roc_auc = metrics.auc(fpr, tpr)
    # method I: plt
    #import matplotlib.pyplot as plt
    #plt.title('Receiver Operating Characteristic')
    #plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    #plt.legend(loc = 'lower right')
    #plt.plot([0, 1], [0, 1],'r--')
    #plt.xlim([-0.05, 1])
    #plt.ylim([0, 1.05])
    #plt.ylabel('True Positive Rate')
    #plt.xlabel('False Positive Rate')
    #plt.show()
    ##################################################
    """
    
elapsed = timeit.default_timer() - start_time #gives total computation time
print("---Run time is %s seconds ---" % elapsed) #prints computation time
print()

print('Accuracy of each fold: \n {}'.format(acc_score))
print()
print('Avg accuracy : \n{}'.format(np.mean(acc_score))); 
print()
print('Std of accuracy : \n{}'.format(np.std(acc_score)))

cm = confusion_matrix(Truth, Output)
print(cm)
print("classification report: ")
print(classification_report(Truth, Output))

sensitivity = cm[0][0]/(cm[0][0]+cm[0][1])
specificity = cm[1][1]/(cm[1][0]+cm[1][1])
precision = (cm[0][0])/(cm[0][0]+cm[1][0])
f1_score = (2*precision*sensitivity)/(precision+sensitivity)

print(sensitivity)
print(specificity)
print(precision)
print(f1_score)

performance_dict['Avg Accuracy'].append(np.mean(acc_score))
performance_dict['Std Accuracy'].append(np.std(acc_score))
performance_dict['Sensitivity'].append(sensitivity)
performance_dict['Specificity'].append(specificity)
performance_dict['Precision'].append(precision)
performance_dict['F1 score'].append(f1_score)
performance_dict['Run time'].append(elapsed)

100%|██████████| 23/23 [03:38<00:00,  9.50s/it]


---Run time is 218.56001792999996 seconds ---

Accuracy of each fold: 
 [0.9167464114832536, 0.9197165323754976, 0.9935143288084465, 0.9467995508141493, 0.9971109157141063, 0.9658394160583942, 0.9067473660614213, 0.9949255197249959, 0.9497287522603979, 0.924164354604307, 0.9712136081125287, 0.9899775358562295, 0.8794117647058823, 0.9879428083653435, 0.9867571594106936, 0.9834836918806384, 0.7824984790103428, 0.985068710359408, 0.9917270963129404, 0.9793113862881305, 0.9963658388855239]

Avg accuracy : 
0.954716725099649

Std of accuracy : 
0.051392930615198076
[[96689  6196]
 [ 2143 65978]]
classification report: 
              precision    recall  f1-score   support

           0       0.98      0.94      0.96    102885
           1       0.91      0.97      0.94     68121

    accuracy                           0.95    171006
   macro avg       0.95      0.95      0.95    171006
weighted avg       0.95      0.95      0.95    171006

0.9397774213928172
0.9685412721480894
0.97831673951

In [9]:
# LightGBM
start_time = timeit.default_timer() #defines start time so computational time can be calculated

acc_score = []
Truth = []
Output = []
 
for X_train, y_train, X_test, y_test in tqdm(selected_splits):
    lgbm = lightgbm.LGBMClassifier(learning_rate=0.09,max_depth=-5,random_state=42)
    
    # Create the LightGBM data containers
    model.fit(X_train,y_train,eval_set=[(X_test,y_test),(X_train,y_train)], verbose=None)

    pred_values = model.predict(X_test)
     
    acc = accuracy_score(y_test, pred_values)
    acc_score.append(acc)
    
    Truth.extend(y_test.values.reshape(y_test.shape[0])); ## it is a list
    Output.extend(pred_values); ## it is a list  
    
    """
    #print(model.feature_importances_)
    Importance = pd.DataFrame({'Importance':(model.feature_importances_*100)[0:10]}, 
                          index = (X_train.columns)[0:10])
    Importance.sort_values(by = 'Importance', 
                       axis = 0, 
                       ascending = True).plot(kind = 'barh', 
                                              color = 'r')
    plt.xlabel('Variable Importance')
    plt.gca().legend_ = None
    #plt.savefig('plot1.png')
    """


elapsed = timeit.default_timer() - start_time #gives total computation time
print("---Run time is %s seconds ---" % elapsed) #prints computation time
print()

print('Accuracy of each fold: \n {}'.format(acc_score))
print()
print('Avg accuracy : \n{}'.format(np.mean(acc_score))); 
print()
print('Std of accuracy : \n{}'.format(np.std(acc_score)))

cm = confusion_matrix(Truth, Output)
print(cm)
print("classification report: ")
print(classification_report(Truth, Output))

sensitivity = cm[0][0]/(cm[0][0]+cm[0][1])
specificity = cm[1][1]/(cm[1][0]+cm[1][1])
precision = (cm[0][0])/(cm[0][0]+cm[1][0])
f1_score = (2*precision*sensitivity)/(precision+sensitivity)

print(sensitivity)
print(specificity)
print(precision)
print(f1_score)

performance_dict['Avg Accuracy'].append(np.mean(acc_score))
performance_dict['Std Accuracy'].append(np.std(acc_score))
performance_dict['Sensitivity'].append(sensitivity)
performance_dict['Specificity'].append(specificity)
performance_dict['Precision'].append(precision)
performance_dict['F1 score'].append(f1_score)
performance_dict['Run time'].append(elapsed)

100%|██████████| 23/23 [04:26<00:00, 11.61s/it]


---Run time is 266.9603945710005 seconds ---

Accuracy of each fold: 
 [0.9167464114832536, 0.9197165323754976, 0.9935143288084465, 0.9467995508141493, 0.9971109157141063, 0.9658394160583942, 0.9067473660614213, 0.9949255197249959, 0.9497287522603979, 0.924164354604307, 0.9712136081125287, 0.9899775358562295, 0.8794117647058823, 0.9037695372356727, 0.7396508728179552, 0.9879428083653435, 0.9867571594106936, 0.9834836918806384, 0.7824984790103428, 0.985068710359408, 0.9917270963129404, 0.9793113862881305, 0.9963658388855239]

Avg accuracy : 
0.9431509407454896

Std of accuracy : 
0.06634500963270624
[[96689  6196]
 [ 5381 79291]]
classification report: 
              precision    recall  f1-score   support

           0       0.95      0.94      0.94    102885
           1       0.93      0.94      0.93     84672

    accuracy                           0.94    187557
   macro avg       0.94      0.94      0.94    187557
weighted avg       0.94      0.94      0.94    187557

0.9397774213